In [27]:
from scipy.ndimage import label, find_objects
from PIL import Image, ImageDraw
import numpy as np
import mss
import cv2

In [2]:
monitor = {
    "top": 0,
    "left": 0,
    "width": 2560,
    "height": 1440,
    "mon": 0,
}

In [63]:
with mss.mss() as sct:
    screenshot1 = sct.grab(monitor)
    mss.tools.to_png(screenshot1.rgb, screenshot1.size, output="test1.png")
    np_screenshot1 = np.flip(np.array(screenshot1, dtype=np.uint8)[:, :, :3], 2)

In [64]:
with mss.mss() as sct:
    screenshot2 = sct.grab(monitor)
    mss.tools.to_png(screenshot2.rgb, screenshot2.size, output="test2.png")
    np_screenshot2 = np.flip(np.array(screenshot2, dtype=np.uint8)[:, :, :3], 2)

In [65]:
diff = np.equal(np_screenshot1, np_screenshot2).all(axis=2).astype(np.uint8)

diff_image = diff.copy()
diff_image[diff_image == 1] = 255
diff_image = Image.fromarray(diff_image)
diff_image.save("test_diff.png","PNG")

In [72]:
# Use floodfill to identify outer shape of objects
imFlood = diff.copy()
h, w = diff.shape[:2]
mask = np.zeros((h+2, w+2), np.uint8)
cv2.floodFill(imFlood, mask, (0,0), 0)
cv2.imwrite('im_3_floodfill.png',imFlood)

# Combine flood filled image with original objects
imFlood[np.where(diff==0)]=255
cv2.imwrite('im_4_mixed_floodfill.png',imFlood)

# Invert output colors
imFlood=~imFlood
cv2.imwrite('im_5_inverted_floodfill.png',imFlood)

source_img = Image.open("test_diff.png").convert("RGB")
draw = ImageDraw.Draw(source_img)
cnts, _ = cv2.findContours(imFlood, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
for cnt in cnts:
    top_x, top_y = cnt[:, :, 0].min(), cnt[:, :, 1].min()
    bot_x, bot_y = cnt[:, :, 0].max(), cnt[:, :, 1].max()
    
    # One of the bounding boxes is going to be the whole image, we don't need it
    # if not (top_x==0 and top_y==0 and bot_x==diff.shape[1]-1 and bot_y==diff.shape[0]-1):
    draw.rectangle(((top_x, top_y), (bot_x, bot_y)), outline ="red")

source_img.save("test_diff_with_boxes.png","PNG")

In [73]:
source_img = Image.open("test_diff.png").convert("RGB")
draw = ImageDraw.Draw(source_img)
cnts, _ = cv2.findContours(diff, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
for cnt in cnts:
    top_x, top_y = cnt[:, :, 0].min(), cnt[:, :, 1].min()
    bot_x, bot_y = cnt[:, :, 0].max(), cnt[:, :, 1].max()
    
    # One of the bounding boxes is going to be the whole image, we don't need it
    # if not (top_x==0 and top_y==0 and bot_x==diff.shape[1]-1 and bot_y==diff.shape[0]-1):
    draw.rectangle(((top_x, top_y), (bot_x, bot_y)), outline ="red")

source_img.save("test_diff_with_boxes.png","PNG")